# ELEC0134 - Applied Machine Learning Systems Assignment 2019/2020
## Task B1: Face Shape Recognition

In this notebook, we are going to detect 5 types of face shapes from the *cartoon_set* dataset:
1. Setup data generator to load images dataset using Keras Image Preprocessing API
2. Build an AlexNet CNN architecture
3. Fit the CNN model to the training data
4. Test the performances of the CNN

In [2]:
# Import libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,accuracy_score,confusion_matrix

%matplotlib inline

### Load labels.csv and convert to pandas dataframe

In [3]:
# Modified the directory
os.chdir('..')
AMLS_dir = os.path.abspath(os.curdir)
basedir = os.path.join(AMLS_dir,'Datasets')
cartoon_dir = os.path.join(basedir,'cartoon_set')
images_dir = os.path.join(cartoon_dir,'img')
labels_path = os.path.join(cartoon_dir,'labels.csv')

# Converting csv into dataframe using read_csv(label_path)
cartoon_df = pd.read_csv(os.path.normcase(labels_path), sep='\t', engine='python')
df = cartoon_df[['file_name', 'face_shape']]
print(df.head())

# Convert the face shape column to class 1-5
df.loc[:,'face_shape'] += 1
print(df.head())

# Convert face shape column type from int64 to str for data generator
df = df.applymap(str)

  file_name  face_shape
0     0.png           4
1     1.png           4
2     2.png           3
3     3.png           0
4     4.png           2
  file_name  face_shape
0     0.png           5
1     1.png           5
2     2.png           4
3     3.png           1
4     4.png           3


C:\Users\User\Anaconda3\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


### 1) Setup data generators to draws from train, test and validation dataframe
1. Convolution Neural Networks fed by batches of images to train, therefore need to convert from (size1, size2, channels) to (samples, size1, size2, channels).
2. Split the dataframe into test, train and validation set.
3. Use Keras Image Processing API (flow_from_dataframe) to load the images dataset according to dataframe

In [4]:
# Import data generator API 
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator

# Setup data generator for test and train dataset generator
train_df, test_df = train_test_split(df, train_size=0.7, random_state=42)
train_datagen = ImageDataGenerator(rescale=1./255,
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                   validation_split=0.2)                     
test_datagen = ImageDataGenerator(rescale=1./255)

# Generating training and validation dataset for AlexNet CNN
train_generator = train_datagen.flow_from_dataframe(dataframe=train_df,
                                                    directory=images_dir,
                                                    x_col="file_name",
                                                    y_col="face_shape",
                                                    target_size=(32, 32),
                                                    batch_size=32,
                                                    shuffle=True,
                                                    class_mode='categorical',
                                                    subset='training')
valid_generator = train_datagen.flow_from_dataframe(dataframe=train_df,
                                                    directory=images_dir,
                                                    x_col="file_name",
                                                    y_col="face_shape",
                                                    target_size=(32, 32),
                                                    batch_size=32,
                                                    shuffle=True, 
                                                    class_mode='categorical',
                                                    subset='validation')

Found 5600 validated image filenames belonging to 5 classes.
Found 1400 validated image filenames belonging to 5 classes.


In [5]:
print(train_generator.image_shape)
print(train_generator.batch_size)

(32, 32, 3)
32


### Using EarlyStopping and ModelCheckpoint Keras API callback to save the weights
Too many training epochs will lead to overfitting the training dataset whereas too few will result underfitting. Therefore, it is better to monitor the performance of the model during training and call EarlyStopping when overfitting occurs and save the weights using ModelCheckpoint

In [6]:
# Use early stopping to terminate training epochs through callbacks
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint

# Seek a mininum for validation loss and display the stopped epochs using verbose and adding delays
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)

# Save best model using checkpoint
B1_dir = os.path.join(AMLS_dir, 'B1')
model_path = os.path.join(B1_dir, 'AlexNet.h5')
mcp = ModelCheckpoint(os.path.normcase(model_path), monitor='val_loss', mode='min', verbose=1, save_best_only=True)

# Define callback function in a list
callback_list = [es, mcp]

### 2) Build the neural network using AlexNet architecture
1. Build the sequential model and add convolutional and max pooling layers to it.
2. Add batch normalization layer between each convolutional layer
3. Add dropout layer between each fully connected layer
4. Add softmax layer at the end
5. Compile the model and use ADAM (Adaptive learning rate optimization algorithm)

In [8]:
# Import network libraries
import keras
from keras.models import Sequential
from keras.utils import to_categorical
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D

# Number of output classes
num_class = 5

# Create a sequential model
model = Sequential()

# Add 1st convolutional layer
model.add(Conv2D(filters=48, input_shape=train_generator.image_shape, kernel_size=(11,11), strides=(4,4), 
                 padding="same", activation = "relu"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding="same"))

# Add 2nd convolutional layer
model.add(Conv2D(filters=128, kernel_size=(5,5), strides=(1,1), padding="same", activation = "relu"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding="same"))

# 3rd Convolutional Layer
model.add(Conv2D(filters=192, kernel_size=(3,3), strides=(1,1), padding="same", activation = "relu"))

# 4th Convolutional Layer
model.add(Conv2D(filters=192, kernel_size=(3,3), strides=(1,1), padding="same", activation = "relu"))

# 5th Convolutional Layer
model.add(Conv2D(filters=128, kernel_size=(3,3), strides=(1,1), padding="same", activation = "relu"))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

# Passing to fully connected layer
model.add(Flatten())

# Add 1st dense layer
model.add(Dense(units=64, activation = "relu"))
model.add(Dropout(0.5))

# Add 2nd dense layer
model.add(Dense(units=64, activation = "relu"))
model.add(Dropout(0.5))

# Output Layer (5 clases for face shapes)
model.add(Dense(units=num_class, activation = "relu"))

# Display summary of the model
model.summary()

# Compile the model using ADAM (Adaptive learning rate optimization)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 8, 8, 48)          17472     
_________________________________________________________________
batch_normalization_3 (Batch (None, 8, 8, 48)          192       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 4, 4, 48)          0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 4, 4, 128)         153728    
_________________________________________________________________
batch_normalization_4 (Batch (None, 4, 4, 128)         512       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 2, 2, 128)         0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 2, 2, 192)        

### 3) Fit the AlexNet CNN to training and validation data
Access model training history using history callback to record training metrics for each epoch

In [ ]:
# Set steps per epoch for callback 
STEP_SIZE_TRAIN = train_generator.samples//train_generator.batch_size
STEP_SIZE_VALID = valid_generator.samples//valid_generator.batch_size
print(STEP_SIZE_TRAIN)
print(STEP_SIZE_VALID)

history = model.fit_generator(generator=train_generator,
                              steps_per_epoch=STEP_SIZE_TRAIN,
                              epochs=30,
                              callbacks=callback_list,
                              validation_data=valid_generator,
                              validation_steps=STEP_SIZE_VALID)

175
43
Epoch 1/30
175/175 [==============================] - 242s 1s/step - loss: nan - accuracy: 0.2018 - val_loss: nan - val_accuracy: 0.1984

Epoch 00001: val_loss did not improve from inf
Epoch 2/30
  3/175 [..............................] - ETA: 7s - loss: nan - accuracy: 0.1979

C:\Users\User\Anaconda3\lib\site-packages\tensorflow_core\python\keras\callbacks.py:1225: RuntimeWarning: invalid value encountered in less
  if self.monitor_op(current - self.min_delta, self.best):
C:\Users\User\Anaconda3\lib\site-packages\tensorflow_core\python\keras\callbacks.py:992: RuntimeWarning: invalid value encountered in less
  if self.monitor_op(current, self.best):


175/175 [==============================] - 25s 142ms/step - loss: nan - accuracy: 0.2018 - val_loss: nan - val_accuracy: 0.1937

Epoch 00002: val_loss did not improve from inf
Epoch 3/30
175/175 [==============================] - 24s 138ms/step - loss: nan - accuracy: 0.2018 - val_loss: nan - val_accuracy: 0.1974

Epoch 00003: val_loss did not improve from inf
Epoch 4/30
175/175 [==============================] - 25s 142ms/step - loss: nan - accuracy: 0.2018 - val_loss: nan - val_accuracy: 0.2025

Epoch 00004: val_loss did not improve from inf
Epoch 5/30
 99/175 [===============>..............] - ETA: 9s - loss: nan - accuracy: 0.2049

### 4) Performances of CNN
1. Plot the learning curve for model validation to select the optimum hyperparameters
2. Evaluating model performances using standard metrics e.g confusion matrix, accurancy score and etc